In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.cluster import KMeans
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [2]:
data = pd.read_csv("D:\\Data\\研赛建模\\B题\\training_set_3ap_loc30_nav82.csv")

In [3]:
data.columns

Index(['test_id', 'test_dur', 'loc_id', 'protocol', 'pkt_len', 'bss_id',
       'ap_name', 'ap_mac', 'ap_id', 'pd', 'ed', 'nav', 'eirp',
       'ap_from_ap_0_sum_ant_rssi', 'ap_from_ap_0_max_ant_rssi',
       'ap_from_ap_0_mean_ant_rssi', 'ap_from_ap_1_sum_ant_rssi',
       'ap_from_ap_1_max_ant_rssi', 'ap_from_ap_1_mean_ant_rssi',
       'ap_from_ap_2_sum_ant_rssi', 'ap_from_ap_2_max_ant_rssi',
       'ap_from_ap_2_mean_ant_rssi', 'sta_mac', 'sta_id',
       'sta_to_ap_0_sum_ant_rssi', 'sta_to_ap_0_max_ant_rssi',
       'sta_to_ap_0_mean_ant_rssi', 'sta_to_ap_1_sum_ant_rssi',
       'sta_to_ap_1_max_ant_rssi', 'sta_to_ap_1_mean_ant_rssi',
       'sta_to_ap_2_sum_ant_rssi', 'sta_to_ap_2_max_ant_rssi',
       'sta_to_ap_2_mean_ant_rssi', 'sta_from_ap_0_sum_ant_rssi',
       'sta_from_ap_0_max_ant_rssi', 'sta_from_ap_0_mean_ant_rssi',
       'sta_from_ap_1_sum_ant_rssi', 'sta_from_ap_1_max_ant_rssi',
       'sta_from_ap_1_mean_ant_rssi', 'sta_from_ap_2_sum_ant_rssi',
       'sta_from_ap_

In [4]:
rssi_columns = [
    'ap_from_ap_0_sum_ant_rssi',
    'ap_from_ap_0_max_ant_rssi',
    'ap_from_ap_0_mean_ant_rssi',
    'ap_from_ap_1_sum_ant_rssi',
    'ap_from_ap_1_max_ant_rssi',
    'ap_from_ap_1_mean_ant_rssi',
    'ap_from_ap_2_sum_ant_rssi', 
    'ap_from_ap_2_max_ant_rssi',
    'ap_from_ap_2_mean_ant_rssi',
    'sta_to_ap_0_sum_ant_rssi',
    'sta_to_ap_0_max_ant_rssi',
    'sta_to_ap_0_mean_ant_rssi',
    'sta_to_ap_1_sum_ant_rssi',
    'sta_to_ap_1_max_ant_rssi',
    'sta_to_ap_1_mean_ant_rssi',
    'sta_to_ap_2_sum_ant_rssi', 
    'sta_to_ap_2_max_ant_rssi',
    'sta_to_ap_2_mean_ant_rssi',
    'sta_from_ap_0_sum_ant_rssi',
    'sta_from_ap_0_max_ant_rssi',
    'sta_from_ap_0_mean_ant_rssi',
    'sta_from_ap_1_sum_ant_rssi',
    'sta_from_ap_1_max_ant_rssi',
    'sta_from_ap_1_mean_ant_rssi',
    'sta_from_ap_2_sum_ant_rssi',
    'sta_from_ap_2_max_ant_rssi', 
    'sta_from_ap_2_mean_ant_rssi'
]

In [5]:
def str_to_float_list(s):
    try:
        return [float(x) for x in ast.literal_eval(s)]
    except:
        return []

In [6]:
for col in rssi_columns:
    data[f"{col}_list"] = data[col].apply(str_to_float_list)

In [7]:
grouped_data = data.groupby('test_id')

In [8]:
def kmeans_filter_rssi(rssi_list, n_clusters=2):
    if len(rssi_list) <= 1:
        return rssi_list
    rssi_array = np.array(rssi_list).reshape(-1, 1)
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    if centers[0] < centers[1]:
        noise_label = 0
    else:
        noise_label = 1
    filtered_rssi = [rssi for rssi, label in zip(rssi_list, labels) if label != noise_label]
    return filtered_rssi
    
    

In [9]:
for test_id, group in grouped_data:
    for col in rssi_columns:
        row_1 = group.iloc[0]
        row_2 = group.iloc[1]
        row_3 = group.iloc[2]
        clean_rssi_row1 = kmeans_filter_rssi(row_1[f"{col}_list"])
        clean_rssi_row2 = kmeans_filter_rssi(row_2[f"{col}_list"])
        clean_rssi_row_3 = kmeans_filter_rssi(row_3[f"{col}_list"])
        data.at[row_1.name, f"{col}_list"] = clean_rssi_row1
        data.at[row_2.name, f"{col}_list"] = clean_rssi_row2
        data.at[row_3.name, f"{col}_list"] = clean_rssi_row_3     

C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\2700997529.py:5: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\2700997529.py:5: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\2700997529.py:5: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\2700997529.py:5: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans

In [10]:
data.head(1)

,test_id,test_dur,loc_id,protocol,pkt_len,bss_id,ap_name,ap_mac,ap_id,pd,ed,nav,eirp,ap_from_ap_0_sum_ant_rssi,ap_from_ap_0_max_ant_rssi,ap_from_ap_0_mean_ant_rssi,ap_from_ap_1_sum_ant_rssi,ap_from_ap_1_max_ant_rssi,ap_from_ap_1_mean_ant_rssi,ap_from_ap_2_sum_ant_rssi,ap_from_ap_2_max_ant_rssi,ap_from_ap_2_mean_ant_rssi,sta_mac,sta_id,sta_to_ap_0_sum_ant_rssi,sta_to_ap_0_max_ant_rssi,sta_to_ap_0_mean_ant_rssi,sta_to_ap_1_sum_ant_rssi,sta_to_ap_1_max_ant_rssi,sta_to_ap_1_mean_ant_rssi,sta_to_ap_2_sum_ant_rssi,sta_to_ap_2_max_ant_rssi,sta_to_ap_2_mean_ant_rssi,sta_from_ap_0_sum_ant_rssi,sta_from_ap_0_max_ant_rssi,sta_from_ap_0_mean_ant_rssi,sta_from_ap_1_sum_ant_rssi,sta_from_ap_1_max_ant_rssi,sta_from_ap_1_mean_ant_rssi,sta_from_ap_2_sum_ant_rssi,sta_from_ap_2_max_ant_rssi,sta_from_ap_2_mean_ant_rssi,sta_from_sta_0_rssi,sta_from_sta_1_rssi,sta_from_sta_2_rssi,nss,mcs,per,num_ampdu,ppdu_dur,other_air_time,seq_time,throughput,predict throughput,error%,ap_from_ap_0_sum_ant_rssi_list,ap_from_ap_0_max_ant_rssi_list,ap_from_ap_0_mean_ant_rssi_list,ap_from_ap_1_sum_ant_rssi_list,ap_from_ap_1_max_ant_rssi_list,ap_from_ap_1_mean_ant_rssi_list,ap_from_ap_2_sum_ant_rssi_list,ap_from_ap_2_max_ant_rssi_list,ap_from_ap_2_mean_ant_rssi_list,sta_to_ap_0_sum_ant_rssi_list,sta_to_ap_0_max_ant_rssi_list,sta_to_ap_0_mean_ant_rssi_list,sta_to_ap_1_sum_ant_rssi_list,sta_to_ap_1_max_ant_rssi_list,sta_to_ap_1_mean_ant_rssi_list,sta_to_ap_2_sum_ant_rssi_list,sta_to_ap_2_max_ant_rssi_list,sta_to_ap_2_mean_ant_rssi_list,sta_from_ap_0_sum_ant_rssi_list,sta_from_ap_0_max_ant_rssi_list,sta_from_ap_0_mean_ant_rssi_list,sta_from_ap_1_sum_ant_rssi_list,sta_from_ap_1_max_ant_rssi_list,sta_from_ap_1_mean_ant_rssi_list,sta_from_ap_2_sum_ant_rssi_list,sta_from_ap_2_max_ant_rssi_list,sta_from_ap_2_mean_ant_rssi_list
0,1,60,loc30,tcp,1500,2,model-20,6c14-6ef5-8e30,ap_2,-82,-62,-82,9,"[-81, -82, -82, -82, -78, -82, -82, -82, -82, ...","[-88, -88, -85, -88, -82, -89, -88, -87, -87, ...","[-90, -91, -90, -91, -87, -91, -91, -91, -91, ...","[-84, -85, -86, -83, -86, -86, -86, -85, -85, ...","[-91, -92, -93, -88, -93, -92, -93, -92, -93, ...","[-93, -94, -95, -92, -95, -95, -95, -94, -94, ...",NaN,NaN,NaN,1287,sta_2,"[-78, -81, -80, -81, -81, -81, -81, -80, -81, ...","[-94, -84, -87, -85, -88, -86, -87, -89, -87, ...","[-87, -90, -89, -90, -90, -90, -91, -89, -90, ...","[-87, -86, -85, -87, -87, -86, -86, -86, -87, ...","[-93, -91, -94, -92, -92, -93, -94, -94, -93, ...","[-96, -95, -94, -96, -96, -95, -95, -95, -96, ...","[-49, -48, -49, -49, -49, -53, -49, -49, -48, ...","[-94, -54, -53, -53, -53, -52, -57, -53, -52, ...","[-58, -57, -58, -58, -58, -60, -58, -58, -57, ...","[-76, -77, -76, -76, -76, -76, -76, -76, -76, ...","[-78, -78, -78, -78, -78, -78, -78, -78, -78, ...","[-80, -80, -79, -79, -79, -79, -79, -79, -79, ...",[-92],[-96],[-98],"[-51, -52, -51, -52, -51, -51, -51, -51, -51, ...","[-53, -54, -53, -54, -53, -53, -53, -53, -53, ...","[-54, -55, -54, -55, -54, -54, -54, -54, -54, ...",-79.0,-78.0,NaN,2,11,0.04,13,0.002424,0.0,49.52,205.32,NaN,NaN,"[-81.0, -78.0, -81.0, -78.0, -81.0, -81.0, -79...","[-85.0, -82.0, -86.0, -83.0, -85.0, -86.0, -83...","[-87.0, -87.0, -87.0, -88.0, -89.0, -88.0]","[-84.0, -83.0, -81.0, -84.0, -82.0, -84.0, -84.0]","[-88.0, -89.0, -84.0, -90.0, -90.0, -87.0, -89...","[-92.0, -90.0, -90.0, -92.0]",[],[],[],"[-78.0, -81.0, -80.0, -81.0, -81.0, -81.0, -81...","[-84.0, -87.0, -85.0, -88.0, -86.0, -87.0, -89...","[-87.0, -90.0, -89.0, -90.0, -90.0, -90.0, -91...","[-79.0, -81.0]","[-81.0, -84.0]","[-87.0, -90.0]","[-49.0, -48.0, -49.0, -49.0, -49.0, -49.0, -49...","[-54.0, -53.0, -53.0, -53.0, -52.0, -57.0, -53...","[-58.0, -57.0, -58.0, -58.0, -58.0, -60.0, -58...","[-76.0, -76.0, -76.0, -76.0, -76.0, -76.0, -76...","[-78.0, -78.0, -78.0, -78.0, -78.0, -78.0, -78...","[-79.0, -79.0, -79.0, -79.0, -79.0, -79.0, -79...",[-92.0],[-96.0],[-98.0],"[-51.0, -51.0, -51.0, -51.0, -51.0, -51.0, -51...","[-53.0, -53.0, -53.0, -53.0, -53.0, -53

In [11]:
def calculate_hearing_probability(rssi_list, threshold):
    if not rssi_list:
        return 0
    above_threshold = [rssi for rssi in rssi_list if rssi >= threshold]
    return len(above_threshold) / len(rssi_list)

# 假设您的数据已经按照'test_id'进行分组
for test_id, group in grouped_data:
    # 检查每个测试是否有3行数据
    if len(group) != 3:
        print(f"测试ID {test_id} 的数据行数不是3行，实际为 {len(group)} 行。请检查数据。")
        continue  # 跳过这个测试

    # 建立一个AP索引到行的映射
    ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}

    # 获取所有的AP ID
    ap_ids = list(ap_rows.keys())  # ['ap_0', 'ap_1', 'ap_2']

    # 为每个AP计算从其他AP接收到的RSSI列表并计算互听概率
    for ap_id in ap_ids:
        current_row = ap_rows[ap_id]
        current_ap_index = int(ap_id.split('_')[1])

        # 获取其他AP的索引
        other_ap_indices = [i for i in range(3) if i != current_ap_index]

        # 初始化一个列表来收集其他AP的RSSI列表
        combined_rssi_list = []

        for other_ap_index in other_ap_indices:
            col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
            rssi_list = current_row[col_name]

            if isinstance(rssi_list, list):
                combined_rssi_list.extend(rssi_list)
            else:
                print(f"测试ID {test_id} 中的AP {ap_id} 没有列 {col_name} 的数据")
                continue  # 跳过没有数据的情况

        # 获取当前AP的PD门限
        pd_threshold = current_row['pd']

        # 计算互听概率
        hearing_probability = calculate_hearing_probability(combined_rssi_list, pd_threshold)

        # 将结果存储在数据中
        data.at[current_row.name, 'ap_hear_prob_pd'] = hearing_probability

    # 计算所有AP之间的最小互听概率
    hearing_probs = [data.at[ap_rows[ap_id].name, 'ap_hear_prob_pd'] for ap_id in ap_ids]
    overall_min_hear_prob = min(hearing_probs)

    # 将最小互听概率存储在所有行中
    for ap_id in ap_ids:
        data.at[ap_rows[ap_id].name, 'ap_pair_hear_prob_pd'] = overall_min_hear_prob


In [12]:
# 假设您已经定义了can_hear_each_other和calculate_nav_period函数

def can_hear_each_other(rssi_list, threshold):
    if not rssi_list or not isinstance(rssi_list, list):
        return False
    return any(rssi >= threshold for rssi in rssi_list)

def calculate_nav_period(seq_time):
    # 您需要根据实际的NAV计算逻辑来实现这个函数
    # 这里我们假设NAV周期等于seq_time
    return seq_time

# 开始处理数据
for test_id, group in grouped_data:
    if len(group) != 3:
        print(f"测试ID {test_id} 的数据行数不是3行，实际为 {len(group)} 行。请检查数据。")
        continue

    # 建立AP ID到数据行的映射
    ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}

    # 获取NAV门限，假设所有AP的NAV门限相同
    nav_threshold = group['nav'].iloc[0]
    print(f"Test ID: {test_id}, NAV Threshold: {nav_threshold}")

    # 初始化每个AP的NAV静默时间和静默比例字典
    nav_silence_times = {}
    nav_silence_ratios = {}

    # 获取测试总时长（假设所有AP的测试时长相同）
    test_duration = group['test_dur'].iloc[0] * 1e6  # 将秒转换为微秒
    print(f"Test Duration (in µs): {test_duration}")

    # 对于每个AP，计算其NAV静默时间和比例
    for ap_id, current_row in ap_rows.items():
        current_ap_index = int(ap_id.split('_')[1])
        current_nav_silence_time = 0

        # 获取其他AP的ID列表
        other_ap_ids = [other_id for other_id in ap_rows.keys() if other_id != ap_id]

        for other_ap_id in other_ap_ids:
            other_row = ap_rows[other_ap_id]
            other_ap_index = int(other_ap_id.split('_')[1])

            # 获取当前AP从其他AP接收到的RSSI列表
            col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
            rssi_list = current_row[col_name]
            print(f"{ap_id} RSSI from {other_ap_id}: {rssi_list}")

            # 判断当前AP是否能听到其他AP
            can_hear = can_hear_each_other(rssi_list, nav_threshold)
            print(f"{ap_id} can hear {other_ap_id}: {can_hear}")

            if can_hear:
                # 获取其他AP的seq_time和num_ampdu
                seq_time_other = other_row['seq_time'] * 1e6  # 将秒转换为微秒
                nav_period = calculate_nav_period(seq_time_other)
                num_transmissions_other = other_row['num_ampdu']
                nav_silence_time = nav_period * num_transmissions_other
                current_nav_silence_time += nav_silence_time
                print(f"{ap_id} NAV silence time from {other_ap_id}: {nav_silence_time}")
            else:
                print(f"{ap_id} cannot hear {other_ap_id}, no NAV silence time added.")

        nav_silence_times[ap_id] = current_nav_silence_time

        # 计算NAV静默比例
        if test_duration != 0:
            nav_silence_ratio = current_nav_silence_time / test_duration
        else:
            nav_silence_ratio = 0
            print("Test duration is 0, silence ratio set to 0.")

        nav_silence_ratios[ap_id] = nav_silence_ratio
        print(f"{ap_id} NAV Silence Ratio: {nav_silence_ratio}")

        # 存储结果
        data.at[current_row.name, f'{ap_id}_nav_silence_time'] = current_nav_silence_time
        data.at[current_row.name, f'{ap_id}_nav_silence_ratio'] = nav_silence_ratio

# 如果需要，将nav_silence_times和nav_silence_ratios添加到data中


Test ID: 1, NAV Threshold: -82
Test Duration (in µs): 60000000.0
ap_2 RSSI from ap_1: [-92.0, -90.0, -90.0, -92.0]
ap_2 can hear ap_1: False
ap_2 cannot hear ap_1, no NAV silence time added.
ap_2 RSSI from ap_0: [-87.0, -87.0, -87.0, -88.0, -89.0, -88.0]
ap_2 can hear ap_0: False
ap_2 cannot hear ap_0, no NAV silence time added.
ap_2 NAV Silence Ratio: 0.0
ap_1 RSSI from ap_2: [-91.0, -89.0, -89.0, -90.0, -90.0, -91.0]
ap_1 can hear ap_2: False
ap_1 cannot hear ap_2, no NAV silence time added.
ap_1 RSSI from ap_0: [-92.0, -93.0, -92.0, -93.0, -92.0, -92.0, -93.0, -93.0, -93.0, -93.0, -93.0, -92.0, -93.0, -93.0, -93.0, -93.0, -93.0, -93.0, -92.0, -93.0, -91.0, -91.0, -93.0, -93.0, -93.0, -93.0, -90.0, -93.0, -93.0, -93.0]
ap_1 can hear ap_0: False
ap_1 cannot hear ap_0, no NAV silence time added.
ap_1 NAV Silence Ratio: 0.0
ap_0 RSSI from ap_2: [-90.0, -90.0, -90.0, -86.0, -90.0, -88.0, -90.0, -90.0, -89.0, -83.0, -87.0, -90.0, -89.0, -90.0, -89.0, -90.0, -87.0, -90.0, -89.0, -90.0, -90

In [13]:
def calculate_pd_counts(rssi_list, pd_threshold):
    if not rssi_list or not isinstance(rssi_list, list):
        return 0
    pd_counts = sum(1 for rssi in rssi_list if rssi >= pd_threshold)
    return pd_counts

# 假设您的数据已经按照'test_id'进行分组
for test_id, group in grouped_data:
    if len(group) != 3:
        print(f"测试ID {test_id} 的数据行数不是3行，实际为 {len(group)} 行。请检查数据。")
        continue

    # 建立AP ID到数据行的映射
    ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}

    # 初始化一个字典来存储每个AP的PD计数
    pd_counts_dict = {}

    # 遍历每个AP
    for ap_id, current_row in ap_rows.items():
        current_ap_index = int(ap_id.split('_')[1])

        # 获取当前AP的PD门限
        pd_threshold = current_row['pd']
        print(f"测试ID {test_id}: {ap_id}的PD门限 = {pd_threshold}")

        # 初始化一个列表来收集当前AP从其他AP接收到的RSSI值
        combined_rssi_list = []

        # 获取其他AP的索引
        other_ap_indices = [i for i in range(3) if i != current_ap_index]

        for other_ap_index in other_ap_indices:
            col_name = f'ap_from_ap_{other_ap_index}_max_ant_rssi_list'
            rssi_list = current_row[col_name]

            print(f"测试ID {test_id}: {ap_id} 从 ap_{other_ap_index} 接收到的RSSI列表 = {rssi_list}")

            if isinstance(rssi_list, list):
                combined_rssi_list.extend(rssi_list)
            else:
                print(f"测试ID {test_id}: {ap_id} 没有列 {col_name} 的数据或数据不是列表类型")
                continue

        # 计算当前AP的PD计数
        pd_counts = calculate_pd_counts(combined_rssi_list, pd_threshold)
        pd_counts_dict[ap_id] = pd_counts

        print(f"测试ID {test_id}: {ap_id} 的PD计数 = {pd_counts}")

        # 将计算结果写入数据框
        data.at[current_row.name, f'{ap_id}_pd_counts'] = pd_counts
        print(f"测试ID {test_id}: 写入数据 - {ap_id}_pd_counts = {pd_counts}")


测试ID 1: ap_2的PD门限 = -82
测试ID 1: ap_2 从 ap_0 接收到的RSSI列表 = [-85.0, -82.0, -86.0, -83.0, -85.0, -86.0, -83.0, -85.0, -82.0, -86.0, -84.0, -84.0]
测试ID 1: ap_2 从 ap_1 接收到的RSSI列表 = [-88.0, -89.0, -84.0, -90.0, -90.0, -87.0, -89.0, -88.0]
测试ID 1: ap_2 的PD计数 = 2
测试ID 1: 写入数据 - ap_2_pd_counts = 2
测试ID 1: ap_1的PD门限 = -82
测试ID 1: ap_1 从 ap_0 接收到的RSSI列表 = [-90.0, -90.0, -88.0, -90.0, -89.0, -89.0, -89.0, -89.0, -90.0, -90.0, -90.0, -90.0, -89.0, -89.0, -90.0, -89.0, -90.0, -90.0, -88.0, -89.0, -87.0, -88.0, -87.0, -90.0, -88.0]
测试ID 1: ap_1 从 ap_2 接收到的RSSI列表 = [-88.0, -86.0, -88.0, -84.0, -86.0, -86.0, -86.0, -89.0]
测试ID 1: ap_1 的PD计数 = 0
测试ID 1: 写入数据 - ap_1_pd_counts = 0
测试ID 1: ap_0的PD门限 = -82
测试ID 1: ap_0 从 ap_1 接收到的RSSI列表 = [-89.0, -86.0, -89.0, -89.0, -87.0, -90.0, -89.0, -90.0, -89.0, -90.0, -89.0, -88.0, -88.0, -89.0, -88.0, -90.0, -86.0, -90.0, -87.0, -89.0, -90.0, -90.0, -90.0, -88.0, -89.0, -89.0, -88.0, -87.0, -84.0, -90.0, -87.0, -89.0, -87.0, -89.0, -88.0, -89.0, -89.0, -90.0, -88.0, 

In [14]:
def list_mean(lst):
    return sum(lst) / len(lst) if lst else None

def list_std(lst):
    return np.std(lst) if lst else None

def get_combined_rssi_list(row):
    current_ap_index = int(row['ap_id'].split('_')[1])
    other_ap_indices = [i for i in range(3) if i != current_ap_index]
    combined_rssi_list = []
    for other_ap_index in other_ap_indices:
        col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
        rssi_list = row[col_name]
        if isinstance(rssi_list, list):
            combined_rssi_list.extend(rssi_list)
    return combined_rssi_list

# 计算每个AP从其他AP接收到的RSSI列表的平均值和标准差
data['mean_rssi_value'] = data.apply(lambda row: list_mean(get_combined_rssi_list(row)), axis=1)
data['std_rssi_value'] = data.apply(lambda row: list_std(get_combined_rssi_list(row)), axis=1)


In [15]:
data['ap_hear_prob_pd']

0      0.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
118    0.981132
119    1.000000
120    1.000000
121    0.981132
122    1.000000
Name: ap_hear_prob_pd, Length: 123, dtype: float64

In [16]:
# 直接根据当前行的'ap_id'，从对应的列中获取NAV静默时间和比例
data['nav_silence_time'] = data.apply(lambda row: row[f"{row['ap_id']}_nav_silence_time"], axis=1)
data['nav_silence_ratio'] = data.apply(lambda row: row[f"{row['ap_id']}_nav_silence_ratio"], axis=1)


In [17]:
# 直接根据当前行的'ap_id'，从对应的列中获取PD计数
data['pd_counts'] = data.apply(lambda row: row[f"{row['ap_id']}_pd_counts"], axis=1)


In [18]:
data

,test_id,test_dur,loc_id,protocol,pkt_len,bss_id,ap_name,ap_mac,ap_id,pd,ed,nav,eirp,ap_from_ap_0_sum_ant_rssi,ap_from_ap_0_max_ant_rssi,ap_from_ap_0_mean_ant_rssi,ap_from_ap_1_sum_ant_rssi,ap_from_ap_1_max_ant_rssi,ap_from_ap_1_mean_ant_rssi,ap_from_ap_2_sum_ant_rssi,ap_from_ap_2_max_ant_rssi,ap_from_ap_2_mean_ant_rssi,sta_mac,sta_id,sta_to_ap_0_sum_ant_rssi,sta_to_ap_0_max_ant_rssi,sta_to_ap_0_mean_ant_rssi,sta_to_ap_1_sum_ant_rssi,sta_to_ap_1_max_ant_rssi,sta_to_ap_1_mean_ant_rssi,sta_to_ap_2_sum_ant_rssi,sta_to_ap_2_max_ant_rssi,sta_to_ap_2_mean_ant_rssi,sta_from_ap_0_sum_ant_rssi,sta_from_ap_0_max_ant_rssi,sta_from_ap_0_mean_ant_rssi,sta_from_ap_1_sum_ant_rssi,sta_from_ap_1_max_ant_rssi,sta_from_ap_1_mean_ant_rssi,sta_from_ap_2_sum_ant_rssi,sta_from_ap_2_max_ant_rssi,sta_from_ap_2_mean_ant_rssi,sta_from_sta_0_rssi,sta_from_sta_1_rssi,sta_from_sta_2_rssi,nss,mcs,per,num_ampdu,ppdu_dur,other_air_time,seq_time,throughput,predict throughput,error%,ap_from_ap_0_sum_ant_rssi_list,ap_from_ap_0_max_ant_rssi_list,ap_from_ap_0_mean_ant_rssi_list,ap_from_ap_1_sum_ant_rssi_list,ap_from_ap_1_max_ant_rssi_list,ap_from_ap_1_mean_ant_rssi_list,ap_from_ap_2_sum_ant_rssi_list,ap_from_ap_2_max_ant_rssi_list,ap_from_ap_2_mean_ant_rssi_list,sta_to_ap_0_sum_ant_rssi_list,sta_to_ap_0_max_ant_rssi_list,sta_to_ap_0_mean_ant_rssi_list,sta_to_ap_1_sum_ant_rssi_list,sta_to_ap_1_max_ant_rssi_list,sta_to_ap_1_mean_ant_rssi_list,sta_to_ap_2_sum_ant_rssi_list,sta_to_ap_2_max_ant_rssi_list,sta_to_ap_2_mean_ant_rssi_list,sta_from_ap_0_sum_ant_rssi_list,sta_from_ap_0_max_ant_rssi_list,sta_from_ap_0_mean_ant_rssi_list,sta_from_ap_1_sum_ant_rssi_list,sta_from_ap_1_max_ant_rssi_list,sta_from_ap_1_mean_ant_rssi_list,sta_from_ap_2_sum_ant_rssi_list,sta_from_ap_2_max_ant_rssi_list,sta_from_ap_2_mean_ant_rssi_list,ap_hear_prob_pd,ap_pair_hear_prob_pd,ap_2_nav_silence_time,ap_2_nav_silence_ratio,ap_1_nav_silence_time,ap_1_nav_silence_ratio,ap_0_nav_silence_time,ap_0_nav_silence_ratio,ap_2_pd_counts,ap_1_pd_counts,ap_0_pd_counts,mean_rssi_value,std_rssi_value,nav_silence_time,nav_silence_ratio,pd_counts
0,1,60,loc30,tcp,1500,2,model-20,6c14-6ef5-8e30,ap_2,-82,-62,-82,9,"[-81, -82, -82, -82, -78, -82, -82, -82, -82, ...","[-88, -88, -85, -88, -82, -89, -88, -87, -87, ...","[-90, -91, -90, -91, -87, -91, -91, -91, -91, ...","[-84, -85, -86, -83, -86, -86, -86, -85, -85, ...","[-91, -92, -93, -88, -93, -92, -93, -92, -93, ...","[-93, -94, -95, -92, -95, -95, -95, -94, -94, ...",NaN,NaN,NaN,1287,sta_2,"[-78, -81, -80, -81, -81, -81, -81, -80, -81, ...","[-94, -84, -87, -85, -88, -86, -87, -89, -87, ...","[-87, -90, -89, -90, -90, -90, -91, -89, -90, ...","[-87, -86, -85, -87, -87, -86, -86, -86, -87, ...","[-93, -91, -94, -92, -92, -93, -94, -94, -93, ...","[-96, -95, -94, -96, -96, -95, -95, -95, -96, ...","[-49, -48, -49, -49, -49, -53, -49, -49, -48, ...","[-94, -54, -53, -53, -53, -52, -57, -53, -52, ...","[-58, -57, -58, -58, -58, -60, -58, -58, -57, ...","[-76, -77, -76, -76, -76, -76, -76, -76, -76, ...","[-78, -78, -78, -78, -78, -78, -78, -78, -78, ...","[-80, -80, -79, -79, -79, -79, -79, -79, -79, ...",[-92],[-96],[-98],"[-51, -52, -51, -52, -51, -51, -51, -51, -51, ...","[-53, -54, -53, -54, -53, -53, -53, -53, -53, ...","[-54, -55, -54, -55, -54, -54, -54, -54, -54, ...",-79.0,-78.0,NaN,2,11,0.04,13,0.002424,0.000000,49.52,205.32,NaN,NaN,"[-81.0, -78.0, -81.0, -78.0, -81.0, -81.0, -79...","[-85.0, -82.0, -86.0, -83.0, -85.0, -86.0, -83...","[-87.0, -87.0, -87.0, -88.0, -89.0, -88.0]","[-84.0, -83.0, -81.0, -84.0, -82.0, -84.0, -84.0]","[-88.0, -89.0, -84.0, -90.0, -90.0, -87.0, -89...","[-92.0, -90.0, -90.0, -92.0]",[],[],[],"[-78.0, -81.0, -80.0, -81.0, -81.0, -81.0, -81...","[-84.0, -87.0, -85.0, -88.0, -86.0, -87.0, -89...","[-87.0, -90.0, -89.0, -90.0, -90.0, -90.0, -91...","[-79.0, -81.0]","[-81.0, -84.0]","[-87.0, -90.0]","[-49.0, -48.0, -49.0, -49.0, -49.0, -49.0, -49...","[-54.0, -53.0, -53.0, -53.0, -52.0, -57.0, -53...","[-58.0, -57.0, -58.0, -58

In [19]:
features = [
    'test_dur', 'protocol', 'pkt_len', 'pd', 'ed', 'nav',
    'eirp', 'nss', 'mcs', 'per', 'mean_rssi_value', 'std_rssi_value', 'ap_hear_prob_pd', 'nav_silence_ratio', 'pd_counts', 'ap_pair_hear_prob_pd'
]

In [20]:
data[features]

,test_dur,protocol,pkt_len,pd,ed,nav,eirp,nss,mcs,per,mean_rssi_value,std_rssi_value,ap_hear_prob_pd,nav_silence_ratio,pd_counts,ap_pair_hear_prob_pd
0,60,tcp,1500,-82,-62,-82,9,2,11,0.04,-89.000000,1.843909,0.000000,0.000000,2.0,0.000000
1,60,tcp,1500,-82,-62,-82,9,2,11,0.04,-92.138889,1.228205,0.000000,0.000000,0.0,0.000000
2,60,tcp,1500,-82,-62,-82,9,2,9,0.08,-91.045455,1.731454,0.000000,0.000000,5.0,0.000000
3,60,udp,1500,-82,-62,-82,9,2,11,0.03,-89.000000,1.843909,0.000000,0.000000,2.0,0.000000
4,60,udp,1500,-82,-62,-82,9,2,11,0.03,-92.138889,1.228205,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,60,tcp,1500,-82,-62,-82,29,2,11,0.04,-77.037736,1.847612,0.981132,11.197333,108.0,0.981132
119,60,tcp,1500,-82,-62,-82,29,2,11,0.05,-73.712121,2.005788,1.000000,15.171833,125.0,0.981132
120,60,udp,1500,-82,-62,-82,29,2,11,0.14,-75.724771,3.147364,1.000000,14.296000,89.0,0.981132
121,60,udp,1500,-82,-62,-82,29,2,11,0.04,-77.037736,1.847612,0.981132,13.246000,108.0,0.981132


In [32]:
import pandas as pd
import numpy as np
import ast
from sklearn.cluster import KMeans

def process_file(file_path):
    # 读取数据
    data = pd.read_csv(file_path)

    # 定义需要处理的列
    rssi_columns = [
        'ap_from_ap_0_sum_ant_rssi',
        'ap_from_ap_0_max_ant_rssi',
        'ap_from_ap_0_mean_ant_rssi',
        'ap_from_ap_1_sum_ant_rssi',
        'ap_from_ap_1_max_ant_rssi',
        'ap_from_ap_1_mean_ant_rssi',
        'ap_from_ap_2_sum_ant_rssi', 
        'ap_from_ap_2_max_ant_rssi',
        'ap_from_ap_2_mean_ant_rssi',
        'sta_to_ap_0_sum_ant_rssi',
        'sta_to_ap_0_max_ant_rssi',
        'sta_to_ap_0_mean_ant_rssi',
        'sta_to_ap_1_sum_ant_rssi',
        'sta_to_ap_1_max_ant_rssi',
        'sta_to_ap_1_mean_ant_rssi',
        'sta_to_ap_2_sum_ant_rssi', 
        'sta_to_ap_2_max_ant_rssi',
        'sta_to_ap_2_mean_ant_rssi',
        'sta_from_ap_0_sum_ant_rssi',
        'sta_from_ap_0_max_ant_rssi',
        'sta_from_ap_0_mean_ant_rssi',
        'sta_from_ap_1_sum_ant_rssi',
        'sta_from_ap_1_max_ant_rssi',
        'sta_from_ap_1_mean_ant_rssi',
        'sta_from_ap_2_sum_ant_rssi',
        'sta_from_ap_2_max_ant_rssi', 
        'sta_from_ap_2_mean_ant_rssi'
    ]

    # 将字符串转换为浮点数列表
    def str_to_float_list(s):
        try:
            return [float(x) for x in ast.literal_eval(s)]
        except:
            return []

    for col in rssi_columns:
        data[f"{col}_list"] = data[col].apply(str_to_float_list)

    grouped_data = data.groupby('test_id')

    # 定义用于去噪的函数
    def kmeans_filter_rssi(rssi_list, n_clusters=2):
        if len(rssi_list) <= 1:
            return rssi_list
        rssi_array = np.array(rssi_list).reshape(-1, 1)
        kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
        labels = kmeans.labels_
        centers = kmeans.cluster_centers_
        if centers[0] < centers[1]:
            noise_label = 0
        else:
            noise_label = 1
        filtered_rssi = [rssi for rssi, label in zip(rssi_list, labels) if label != noise_label]
        return filtered_rssi

    # 对每个测试ID的每个RSSI列进行去噪处理
    for test_id, group in grouped_data:
        if len(group) != 3:
            print(f"测试ID {test_id} 的数据行数不是3行，实际为 {len(group)} 行。请检查数据。")
            continue  # 跳过这个测试
        for col in rssi_columns:
            for idx, row in group.iterrows():
                clean_rssi = kmeans_filter_rssi(row[f"{col}_list"])
                data.at[idx, f"{col}_list"] = clean_rssi

    # 定义计算互听概率的函数
    def calculate_hearing_probability(rssi_list, threshold):
        if not rssi_list:
            return 0
        above_threshold = [rssi for rssi in rssi_list if rssi >= threshold]
        return len(above_threshold) / len(rssi_list)

    # 计算每个AP的互听概率
    for test_id, group in grouped_data:
        if len(group) != 3:
            continue  # 已经在上面提示过，这里直接跳过
        ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}
        ap_ids = list(ap_rows.keys())  # ['ap_0', 'ap_1', 'ap_2']

        for ap_id in ap_ids:
            current_row = ap_rows[ap_id]
            current_ap_index = int(ap_id.split('_')[1])
            other_ap_indices = [i for i in range(3) if i != current_ap_index]
            combined_rssi_list = []

            for other_ap_index in other_ap_indices:
                col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
                rssi_list = current_row[col_name]

                if isinstance(rssi_list, list):
                    combined_rssi_list.extend(rssi_list)

            pd_threshold = current_row['pd']
            hearing_probability = calculate_hearing_probability(combined_rssi_list, pd_threshold)
            data.at[current_row.name, 'ap_hear_prob_pd'] = hearing_probability

        hearing_probs = [data.at[ap_rows[ap_id].name, 'ap_hear_prob_pd'] for ap_id in ap_ids]
        overall_min_hear_prob = min(hearing_probs)

        for ap_id in ap_ids:
            data.at[ap_rows[ap_id].name, 'ap_pair_hear_prob_pd'] = overall_min_hear_prob

    # 定义辅助函数
    def can_hear_each_other(rssi_list, threshold):
        if not rssi_list or not isinstance(rssi_list, list):
            return False
        return any(rssi >= threshold for rssi in rssi_list)

    def calculate_nav_period(seq_time):
        # 您需要根据实际的NAV计算逻辑来实现这个函数
        # 这里我们假设NAV周期等于seq_time
        return seq_time

    # 计算每个AP的NAV静默时间和比例
    for test_id, group in grouped_data:
        if len(group) != 3:
            continue
        ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}
        nav_threshold = group['nav'].iloc[0]
        test_duration = group['test_dur'].iloc[0] * 1e6  # 将秒转换为微秒

        for ap_id, current_row in ap_rows.items():
            current_ap_index = int(ap_id.split('_')[1])
            current_nav_silence_time = 0
            other_ap_ids = [other_id for other_id in ap_rows.keys() if other_id != ap_id]

            for other_ap_id in other_ap_ids:
                other_row = ap_rows[other_ap_id]
                other_ap_index = int(other_ap_id.split('_')[1])
                col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
                rssi_list = current_row[col_name]

                can_hear = can_hear_each_other(rssi_list, nav_threshold)

                if can_hear:
                    seq_time_other = other_row['seq_time'] * 1e6  # 将秒转换为微秒
                    nav_period = calculate_nav_period(seq_time_other)
                    num_transmissions_other = other_row['num_ampdu']
                    nav_silence_time = nav_period * num_transmissions_other
                    current_nav_silence_time += nav_silence_time

            if test_duration != 0:
                nav_silence_ratio = current_nav_silence_time / test_duration
            else:
                nav_silence_ratio = 0

            data.at[current_row.name, f'{ap_id}_nav_silence_time'] = current_nav_silence_time
            data.at[current_row.name, f'{ap_id}_nav_silence_ratio'] = nav_silence_ratio

    # 计算每个AP的PD计数
    def calculate_pd_counts(rssi_list, pd_threshold):
        if not rssi_list or not isinstance(rssi_list, list):
            return 0
        pd_counts = sum(1 for rssi in rssi_list if rssi >= pd_threshold)
        return pd_counts

    for test_id, group in grouped_data:
        if len(group) != 3:
            continue
        ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}
        pd_counts_dict = {}

        for ap_id, current_row in ap_rows.items():
            current_ap_index = int(ap_id.split('_')[1])
            pd_threshold = current_row['pd']
            combined_rssi_list = []
            other_ap_indices = [i for i in range(3) if i != current_ap_index]

            for other_ap_index in other_ap_indices:
                col_name = f'ap_from_ap_{other_ap_index}_max_ant_rssi_list'
                rssi_list = current_row[col_name]

                if isinstance(rssi_list, list):
                    combined_rssi_list.extend(rssi_list)

            pd_counts = calculate_pd_counts(combined_rssi_list, pd_threshold)
            data.at[current_row.name, f'{ap_id}_pd_counts'] = pd_counts

    # 计算平均RSSI值和标准差
    def list_mean(lst):
        return sum(lst) / len(lst) if lst else None

    def list_std(lst):
        return np.std(lst) if lst else None

    def get_combined_rssi_list(row):
        current_ap_index = int(row['ap_id'].split('_')[1])
        other_ap_indices = [i for i in range(3) if i != current_ap_index]
        combined_rssi_list = []
        for other_ap_index in other_ap_indices:
            col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
            rssi_list = row[col_name]
            if isinstance(rssi_list, list):
                combined_rssi_list.extend(rssi_list)
        return combined_rssi_list

    data['mean_rssi_value'] = data.apply(lambda row: list_mean(get_combined_rssi_list(row)), axis=1)
    data['std_rssi_value'] = data.apply(lambda row: list_std(get_combined_rssi_list(row)), axis=1)

    # 获取NAV静默时间和比例
    data['nav_silence_time'] = data.apply(lambda row: row.get(f"{row['ap_id']}_nav_silence_time", 0), axis=1)
    data['nav_silence_ratio'] = data.apply(lambda row: row.get(f"{row['ap_id']}_nav_silence_ratio", 0), axis=1)

    # 获取PD计数
    data['pd_counts'] = data.apply(lambda row: row.get(f"{row['ap_id']}_pd_counts", 0), axis=1)

    # 选择需要的特征
    features = [
        'test_dur', 'protocol', 'pkt_len', 'pd', 'ed', 'nav',
        'eirp', 'nss', 'mcs', 'per', 'mean_rssi_value', 'std_rssi_value',
        'ap_hear_prob_pd', 'nav_silence_ratio', 'pd_counts', 'ap_pair_hear_prob_pd',
        'throughput'  # 如果您需要目标变量
    ]

    # 返回处理后的数据
    return data[features]

# 主程序部分
if __name__ == "__main__":
    # 文件列表
    file_list = [
        "D:\\Data\\研赛建模\\B题\\training_set_3ap_loc30_nav82.csv",
        "D:\\Data\\研赛建模\\B题\\training_set_3ap_loc30_nav86.csv",
        "D:\\Data\\研赛建模\\B题\\training_set_3ap_loc31_nav82.csv",
        "D:\\Data\\研赛建模\\B题\\training_set_3ap_loc31_nav86.csv",
        "D:\\Data\\研赛建模\\B题\\training_set_3ap_loc32_nav82.csv",
        "D:\\Data\\研赛建模\B题\\training_set_3ap_loc32_nav86.csv",
        "D:\\Data\\研赛建模\\B题\\training_set_3ap_loc33_nav82.csv",
        "D:\\Data\\研赛建模\B题\\training_set_3ap_loc33_nav88.csv"
    ]

    # 存储处理后的数据
    processed_data_list = []

    # 处理每个文件
    for file_path in file_list:
        print(f"正在处理文件：{file_path}")
        processed_data = process_file(file_path)
        processed_data_list.append(processed_data)

    # 合并所有数据集
    combined_data = pd.concat(processed_data_list, ignore_index=True)

    # 保存合并后的数据集到CSV文件（可选）
    combined_data.to_csv("combined_training_set.csv", index=False)

    # 输出处理完成的提示
    print("所有文件已处理完成，合并后的数据集已生成。")


正在处理文件：D:\Data\研赛建模\B题\training_set_3ap_loc30_nav82.csv


C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KM

正在处理文件：D:\Data\研赛建模\B题\training_set_3ap_loc30_nav86.csv


C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KM

正在处理文件：D:\Data\研赛建模\B题\training_set_3ap_loc31_nav82.csv


...

正在处理文件：D:\Data\研赛建模\B题\training_set_3ap_loc33_nav82.csv


C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KM

正在处理文件：D:\Data\研赛建模\B题\training_set_3ap_loc33_nav88.csv


C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\1835329522.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KM

所有文件已处理完成，合并后的数据集已生成。


In [33]:
train_data = pd.read_csv("D:\Data\研赛建模\B题\combined_training_set.csv")

In [34]:
train_data

,test_dur,protocol,pkt_len,pd,ed,nav,eirp,nss,mcs,per,mean_rssi_value,std_rssi_value,ap_hear_prob_pd,nav_silence_ratio,pd_counts,ap_pair_hear_prob_pd,throughput
0,60,tcp,1500,-82,-62,-82,9,2,11,0.04,-89.000000,1.843909,0.000000,0.000000,2.0,0.000000,205.32
1,60,tcp,1500,-82,-62,-82,9,2,11,0.04,-92.138889,1.228205,0.000000,0.000000,0.0,0.000000,206.67
2,60,tcp,1500,-82,-62,-82,9,2,9,0.08,-91.045455,1.731454,0.000000,0.000000,5.0,0.000000,144.00
3,60,udp,1500,-82,-62,-82,9,2,11,0.03,-89.000000,1.843909,0.000000,0.000000,2.0,0.000000,199.63
4,60,udp,1500,-82,-62,-82,9,2,11,0.03,-92.138889,1.228205,0.000000,0.000000,0.0,0.000000,161.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,60,tcp,1500,-82,-62,-88,29,2,11,0.17,-77.163291,2.634228,0.992405,8.312000,799.0,0.992405,58.67
854,60,tcp,1500,-82,-62,-88,29,2,11,0.09,-75.185372,2.184939,0.993695,5.488000,810.0,0.992405,86.66
855,60,udp,1500,-82,-62,-88,29,2,8,0.27,-75.658824,2.991481,1.000000,14.038167,596.0,0.992405,27.35
856,60,udp,1500,-82,-62,-88,29,2,11,0.28,-77.163291,2.634228,0.992405,8.691667,799.0,0.992405,95.90


In [35]:
features = [
        'test_dur', 'protocol', 'pkt_len', 'pd', 'ed', 'nav',
        'eirp', 'nss', 'mcs', 'per', 'mean_rssi_value', 'std_rssi_value',
        'ap_hear_prob_pd', 'nav_silence_ratio', 'pd_counts', 'ap_pair_hear_prob_pd', # 如果您需要目标变量
    ]

In [36]:
train_data

,test_dur,protocol,pkt_len,pd,ed,nav,eirp,nss,mcs,per,mean_rssi_value,std_rssi_value,ap_hear_prob_pd,nav_silence_ratio,pd_counts,ap_pair_hear_prob_pd,throughput
0,60,tcp,1500,-82,-62,-82,9,2,11,0.04,-89.000000,1.843909,0.000000,0.000000,2.0,0.000000,205.32
1,60,tcp,1500,-82,-62,-82,9,2,11,0.04,-92.138889,1.228205,0.000000,0.000000,0.0,0.000000,206.67
2,60,tcp,1500,-82,-62,-82,9,2,9,0.08,-91.045455,1.731454,0.000000,0.000000,5.0,0.000000,144.00
3,60,udp,1500,-82,-62,-82,9,2,11,0.03,-89.000000,1.843909,0.000000,0.000000,2.0,0.000000,199.63
4,60,udp,1500,-82,-62,-82,9,2,11,0.03,-92.138889,1.228205,0.000000,0.000000,0.0,0.000000,161.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,60,tcp,1500,-82,-62,-88,29,2,11,0.17,-77.163291,2.634228,0.992405,8.312000,799.0,0.992405,58.67
854,60,tcp,1500,-82,-62,-88,29,2,11,0.09,-75.185372,2.184939,0.993695,5.488000,810.0,0.992405,86.66
855,60,udp,1500,-82,-62,-88,29,2,8,0.27,-75.658824,2.991481,1.000000,14.038167,596.0,0.992405,27.35
856,60,udp,1500,-82,-62,-88,29,2,11,0.28,-77.163291,2.634228,0.992405,8.691667,799.0,0.992405,95.90


In [37]:
train_data.replace([np.inf, -np.inf], np.nan, inplace=True)
train_data.dropna(inplace=True)

In [38]:
X = train_data[features]
y = train_data['throughput']

In [39]:
X

,test_dur,protocol,pkt_len,pd,ed,nav,eirp,nss,mcs,per,mean_rssi_value,std_rssi_value,ap_hear_prob_pd,nav_silence_ratio,pd_counts,ap_pair_hear_prob_pd
0,60,tcp,1500,-82,-62,-82,9,2,11,0.04,-89.000000,1.843909,0.000000,0.000000,2.0,0.000000
1,60,tcp,1500,-82,-62,-82,9,2,11,0.04,-92.138889,1.228205,0.000000,0.000000,0.0,0.000000
2,60,tcp,1500,-82,-62,-82,9,2,9,0.08,-91.045455,1.731454,0.000000,0.000000,5.0,0.000000
3,60,udp,1500,-82,-62,-82,9,2,11,0.03,-89.000000,1.843909,0.000000,0.000000,2.0,0.000000
4,60,udp,1500,-82,-62,-82,9,2,11,0.03,-92.138889,1.228205,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853,60,tcp,1500,-82,-62,-88,29,2,11,0.17,-77.163291,2.634228,0.992405,8.312000,799.0,0.992405
854,60,tcp,1500,-82,-62,-88,29,2,11,0.09,-75.185372,2.184939,0.993695,5.488000,810.0,0.992405
855,60,udp,1500,-82,-62,-88,29,2,8,0.27,-75.658824,2.991481,1.000000,14.038167,596.0,0.992405
856,60,udp,1500,-82,-62,-88,29,2,11,0.28,-77.163291,2.634228,0.992405,8.691667,799.0,0.992405


In [40]:
X = pd.get_dummies(X, columns=['protocol'], drop_first=True)
# X.replace([np.inf, -np.inf], np.nan, inplace=True)
# X.dropna(inplace=True)

In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
rf_model = RandomForestRegressor(n_estimators=1000, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
rmse = mse ** 0.5
print(f'Root Mean Squared Error: {rmse}')

Mean Squared Error: 390.9745697145255
Root Mean Squared Error: 19.77307689042162


In [44]:
import xgboost as xgb
xgb_model = xgb.XGBRegressor(n_estimators=1000, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}, MSE: {mse}, R^2: {r2}')

MAE: 13.828074896594126, MSE: 393.8905746446744, R^2: 0.8340181423645323


In [77]:

import pandas as pd

# 创建初始数据
data = {
    'mcs': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]*2,
    'nss': [1]*12 + [2]*12,
    '值': [8.6, 17.2, 25.8, 34.4, 51.6, 68.8, 77.4, 86.0, 103.2, 114.7, 129.0, 143.4,
           17.2, 34.4, 51.6, 68.8, 103.2, 137.6, 154.9, 172.1, 206.5, 229.4, 258.1, 286.8]
}

df = pd.DataFrame(data)

# 去除重复的值并排序
unique_values = df['值'].unique()
unique_values.sort()

# 创建值到编号的映射
value_to_id = {v: idx+1 for idx, v in enumerate(unique_values)}

# 添加编号列
df['编号'] = df['值'].map(value_to_id)

print(df)


    mcs  nss      值  编号
0     0    1    8.6   1
1     1    1   17.2   2
2     2    1   25.8   3
3     3    1   34.4   4
4     4    1   51.6   5
5     5    1   68.8   6
6     6    1   77.4   7
7     7    1   86.0   8
8     8    1  103.2   9
9     9    1  114.7  10
10   10    1  129.0  11
11   11    1  143.4  13
12    0    2   17.2   2
13    1    2   34.4   4
14    2    2   51.6   5
15    3    2   68.8   6
16    4    2  103.2   9
17    5    2  137.6  12
18    6    2  154.9  14
19    7    2  172.1  15
20    8    2  206.5  16
21    9    2  229.4  17
22   10    2  258.1  18
23   11    2  286.8  19


In [78]:
X = X.merge(df[['mcs', 'nss', '值', '编号']], on=['mcs', 'nss'], how='left')


In [79]:
X[X['值'].isna()]

,test_dur,pkt_len,pd,ed,nav,eirp,nss,mcs,per,mean_rssi_value,std_rssi_value,ap_hear_prob_pd,nav_silence_ratio,pd_counts,ap_pair_hear_prob_pd,protocol_udp,值,编号
654,60,1500,-82,-62,-82,10,0,0,0.18,-81.271186,4.722218,0.508475,24.650667,62.0,0.508475,0,NaN,NaN
697,60,1500,-82,-62,-82,20,0,0,0.32,-81.017500,1.801303,0.896250,16.458667,803.0,0.000000,1,NaN,NaN


In [80]:
X = X.drop(index=[654, 697])

In [83]:
features_2 = ['test_dur', 'pkt_len', 'pd', 'ed', 'nav', 'eirp', 'per', 'mean_rssi_value', 
              'std_rssi_value', 'nav_silence_ratio', 'pd_counts','ap_pair_hear_prob_pd', 'protocol_udp']

In [84]:
X_2 = X[features_2]
y_2 = X['编号']
y_2 = y_2.astype(int)

In [85]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)

In [86]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_2, y_train_2)
y_pred_encoded = rf_classifier.predict(X_test_2)

from sklearn.metrics import accuracy_score, classification_report

# 计算准确率
accuracy = accuracy_score(y_test_2, y_pred_encoded)
print("模型准确率:", accuracy)
print("Classification Report:")
print(classification_report(y_test_2, y_pred_encoded))

# (3) 混淆矩阵
print("Confusion Matrix:")
print(confusion_matrix(y_test_2, y_pred_encoded))


模型准确率: 0.7048192771084337
Classification Report:
              precision    recall  f1-score   support

           6       1.00      0.20      0.33         5
           9       0.61      0.89      0.72        19
          10       0.00      0.00      0.00         1
          12       0.60      0.75      0.67        16
          14       0.60      0.60      0.60         5
          15       0.67      0.40      0.50        15
          16       0.56      0.38      0.45        13
          17       0.50      0.20      0.29        15
          18       0.50      0.25      0.33         8
          19       0.81      0.99      0.89        69

    accuracy                           0.70       166
   macro avg       0.58      0.47      0.48       166
weighted avg       0.69      0.70      0.67       166

Confusion Matrix:
[[ 1  4  0  0  0  0  0  0  0  0]
 [ 0 17  0  2  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  1]
 [ 0  1  0 12  1  1  0  0  0  1]
 [ 0  0  0  0  3  0  0  1  0  1]
 [ 0  1  

d:\Users\ROG\anaconda3\envs\calf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Users\ROG\anaconda3\envs\calf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\Users\ROG\anaconda3\envs\calf\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
test_data = pd.read_csv("D:\\Data\\研赛建模\\B题\\test_set_1_3ap.csv")

In [46]:
import pandas as pd
import numpy as np
import ast
from sklearn.cluster import KMeans

def process_file(file_path):
    # 读取数据
    data = pd.read_csv(file_path)

    # 定义需要处理的列
    rssi_columns = [
        'ap_from_ap_0_sum_ant_rssi',
        'ap_from_ap_0_max_ant_rssi',
        'ap_from_ap_0_mean_ant_rssi',
        'ap_from_ap_1_sum_ant_rssi',
        'ap_from_ap_1_max_ant_rssi',
        'ap_from_ap_1_mean_ant_rssi',
        'ap_from_ap_2_sum_ant_rssi', 
        'ap_from_ap_2_max_ant_rssi',
        'ap_from_ap_2_mean_ant_rssi',
        'sta_to_ap_0_sum_ant_rssi',
        'sta_to_ap_0_max_ant_rssi',
        'sta_to_ap_0_mean_ant_rssi',
        'sta_to_ap_1_sum_ant_rssi',
        'sta_to_ap_1_max_ant_rssi',
        'sta_to_ap_1_mean_ant_rssi',
        'sta_to_ap_2_sum_ant_rssi', 
        'sta_to_ap_2_max_ant_rssi',
        'sta_to_ap_2_mean_ant_rssi',
        'sta_from_ap_0_sum_ant_rssi',
        'sta_from_ap_0_max_ant_rssi',
        'sta_from_ap_0_mean_ant_rssi',
        'sta_from_ap_1_sum_ant_rssi',
        'sta_from_ap_1_max_ant_rssi',
        'sta_from_ap_1_mean_ant_rssi',
        'sta_from_ap_2_sum_ant_rssi',
        'sta_from_ap_2_max_ant_rssi', 
        'sta_from_ap_2_mean_ant_rssi'
    ]

    # 将字符串转换为浮点数列表
    def str_to_float_list(s):
        try:
            return [float(x) for x in ast.literal_eval(s)]
        except:
            return []

    for col in rssi_columns:
        data[f"{col}_list"] = data[col].apply(str_to_float_list)

    grouped_data = data.groupby('test_id')

    # 定义用于去噪的函数
    def kmeans_filter_rssi(rssi_list, n_clusters=2):
        if len(rssi_list) <= 1:
            return rssi_list
        rssi_array = np.array(rssi_list).reshape(-1, 1)
        kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
        labels = kmeans.labels_
        centers = kmeans.cluster_centers_
        if centers[0] < centers[1]:
            noise_label = 0
        else:
            noise_label = 1
        filtered_rssi = [rssi for rssi, label in zip(rssi_list, labels) if label != noise_label]
        return filtered_rssi

    # 对每个测试ID的每个RSSI列进行去噪处理
    for test_id, group in grouped_data:
        if len(group) != 3:
            print(f"测试ID {test_id} 的数据行数不是3行，实际为 {len(group)} 行。请检查数据。")
            continue  # 跳过这个测试
        for col in rssi_columns:
            for idx, row in group.iterrows():
                clean_rssi = kmeans_filter_rssi(row[f"{col}_list"])
                data.at[idx, f"{col}_list"] = clean_rssi

    # 定义计算互听概率的函数
    def calculate_hearing_probability(rssi_list, threshold):
        if not rssi_list:
            return 0
        above_threshold = [rssi for rssi in rssi_list if rssi >= threshold]
        return len(above_threshold) / len(rssi_list)

    # 计算每个AP的互听概率
    for test_id, group in grouped_data:
        if len(group) != 3:
            continue  # 已经在上面提示过，这里直接跳过
        ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}
        ap_ids = list(ap_rows.keys())  # ['ap_0', 'ap_1', 'ap_2']

        for ap_id in ap_ids:
            current_row = ap_rows[ap_id]
            current_ap_index = int(ap_id.split('_')[1])
            other_ap_indices = [i for i in range(3) if i != current_ap_index]
            combined_rssi_list = []

            for other_ap_index in other_ap_indices:
                col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
                rssi_list = current_row[col_name]

                if isinstance(rssi_list, list):
                    combined_rssi_list.extend(rssi_list)

            pd_threshold = current_row['pd']
            hearing_probability = calculate_hearing_probability(combined_rssi_list, pd_threshold)
            data.at[current_row.name, 'ap_hear_prob_pd'] = hearing_probability

        hearing_probs = [data.at[ap_rows[ap_id].name, 'ap_hear_prob_pd'] for ap_id in ap_ids]
        overall_min_hear_prob = min(hearing_probs)

        for ap_id in ap_ids:
            data.at[ap_rows[ap_id].name, 'ap_pair_hear_prob_pd'] = overall_min_hear_prob

    # 定义辅助函数
    def can_hear_each_other(rssi_list, threshold):
        if not rssi_list or not isinstance(rssi_list, list):
            return False
        return any(rssi >= threshold for rssi in rssi_list)

    def calculate_nav_period(seq_time):
        # 您需要根据实际的NAV计算逻辑来实现这个函数
        # 这里我们假设NAV周期等于seq_time
        return seq_time

    # 计算每个AP的NAV静默时间和比例
    for test_id, group in grouped_data:
        if len(group) != 3:
            continue
        ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}
        nav_threshold = group['nav'].iloc[0]
        test_duration = group['test_dur'].iloc[0] * 1e6  # 将秒转换为微秒

        for ap_id, current_row in ap_rows.items():
            current_ap_index = int(ap_id.split('_')[1])
            current_nav_silence_time = 0
            other_ap_ids = [other_id for other_id in ap_rows.keys() if other_id != ap_id]

            for other_ap_id in other_ap_ids:
                other_row = ap_rows[other_ap_id]
                other_ap_index = int(other_ap_id.split('_')[1])
                col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
                rssi_list = current_row[col_name]

                can_hear = can_hear_each_other(rssi_list, nav_threshold)

                if can_hear:
                    seq_time_other = other_row['seq_time'] * 1e6  # 将秒转换为微秒
                    nav_period = calculate_nav_period(seq_time_other)
                    num_transmissions_other = other_row['num_ampdu']
                    nav_silence_time = nav_period * num_transmissions_other
                    current_nav_silence_time += nav_silence_time

            if test_duration != 0:
                nav_silence_ratio = current_nav_silence_time / test_duration
            else:
                nav_silence_ratio = 0

            data.at[current_row.name, f'{ap_id}_nav_silence_time'] = current_nav_silence_time
            data.at[current_row.name, f'{ap_id}_nav_silence_ratio'] = nav_silence_ratio

    # 计算每个AP的PD计数
    def calculate_pd_counts(rssi_list, pd_threshold):
        if not rssi_list or not isinstance(rssi_list, list):
            return 0
        pd_counts = sum(1 for rssi in rssi_list if rssi >= pd_threshold)
        return pd_counts

    for test_id, group in grouped_data:
        if len(group) != 3:
            continue
        ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}
        pd_counts_dict = {}

        for ap_id, current_row in ap_rows.items():
            current_ap_index = int(ap_id.split('_')[1])
            pd_threshold = current_row['pd']
            combined_rssi_list = []
            other_ap_indices = [i for i in range(3) if i != current_ap_index]

            for other_ap_index in other_ap_indices:
                col_name = f'ap_from_ap_{other_ap_index}_max_ant_rssi_list'
                rssi_list = current_row[col_name]

                if isinstance(rssi_list, list):
                    combined_rssi_list.extend(rssi_list)

            pd_counts = calculate_pd_counts(combined_rssi_list, pd_threshold)
            data.at[current_row.name, f'{ap_id}_pd_counts'] = pd_counts

    # 计算平均RSSI值和标准差
    def list_mean(lst):
        return sum(lst) / len(lst) if lst else None

    def list_std(lst):
        return np.std(lst) if lst else None

    def get_combined_rssi_list(row):
        current_ap_index = int(row['ap_id'].split('_')[1])
        other_ap_indices = [i for i in range(3) if i != current_ap_index]
        combined_rssi_list = []
        for other_ap_index in other_ap_indices:
            col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
            rssi_list = row[col_name]
            if isinstance(rssi_list, list):
                combined_rssi_list.extend(rssi_list)
        return combined_rssi_list

    data['mean_rssi_value'] = data.apply(lambda row: list_mean(get_combined_rssi_list(row)), axis=1)
    data['std_rssi_value'] = data.apply(lambda row: list_std(get_combined_rssi_list(row)), axis=1)

    # 获取NAV静默时间和比例
    data['nav_silence_time'] = data.apply(lambda row: row.get(f"{row['ap_id']}_nav_silence_time", 0), axis=1)
    data['nav_silence_ratio'] = data.apply(lambda row: row.get(f"{row['ap_id']}_nav_silence_ratio", 0), axis=1)

    # 获取PD计数
    data['pd_counts'] = data.apply(lambda row: row.get(f"{row['ap_id']}_pd_counts", 0), axis=1)

    # 选择需要的特征
    features = [
        'test_dur', 'protocol', 'pkt_len', 'pd', 'ed', 'nav',
        'eirp', 'nss', 'mcs', 'per', 'mean_rssi_value', 'std_rssi_value',
        'ap_hear_prob_pd', 'nav_silence_ratio', 'pd_counts', 'ap_pair_hear_prob_pd',
        'throughput'  # 如果您需要目标变量
    ]

    # 返回处理后的数据
    return data[features]

# 主程序部分
if __name__ == "__main__":
    # 文件列表
    file_list = [
        "D:\\Data\\研赛建模\\B题\\test_set_1_3ap.csv"
    ]

    # 存储处理后的数据
    processed_data_list = []

    # 处理每个文件
    for file_path in file_list:
        print(f"正在处理文件：{file_path}")
        processed_data = process_file(file_path)
        processed_data_list.append(processed_data)

    # 合并所有数据集
    combined_data = pd.concat(processed_data_list, ignore_index=True)

    # 保存合并后的数据集到CSV文件（可选）
    combined_data.to_csv("test_training_set.csv", index=False)

    # 输出处理完成的提示
    print("所有文件已处理完成，合并后的数据集已生成。")


正在处理文件：D:\Data\研赛建模\B题\test_set_1_3ap.csv


C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\2379984645.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\2379984645.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\2379984645.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_51580\2379984645.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KM

所有文件已处理完成，合并后的数据集已生成。


In [47]:
test_data = pd.read_csv("D:\\Data\\研赛建模\\B题\\test_training_set.csv")

In [48]:
test_data

,test_dur,protocol,pkt_len,pd,ed,nav,eirp,nss,mcs,per,mean_rssi_value,std_rssi_value,ap_hear_prob_pd,nav_silence_ratio,pd_counts,ap_pair_hear_prob_pd,throughput
0,60,tcp,1500,-82,-62,-86,9,2,3,0.14,-82.813953,3.696358,0.348837,NaN,79.0,0.293103,NaN
1,60,tcp,1500,-82,-62,-86,9,2,4,0.13,-84.086207,4.260017,0.293103,NaN,42.0,0.293103,NaN
2,60,tcp,1500,-82,-62,-86,9,2,4,0.15,-79.225000,4.803579,0.825000,NaN,41.0,0.293103,NaN
3,60,udp,1500,-82,-62,-86,9,1,5,0.15,-82.813953,3.696358,0.348837,NaN,79.0,0.293103,NaN
4,60,udp,1500,-82,-62,-86,9,2,4,0.13,-84.086207,4.260017,0.293103,NaN,42.0,0.293103,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,60,tcp,1500,-82,-62,-86,29,2,11,0.19,-76.277778,2.841024,0.988372,NaN,794.0,0.988372,NaN
101,60,tcp,1500,-82,-62,-86,29,2,11,0.15,-73.325758,2.142627,0.997475,NaN,800.0,0.988372,NaN
102,60,udp,1500,-82,-62,-86,29,2,8,0.33,-74.237221,3.802473,1.000000,NaN,789.0,0.988372,NaN
103,60,udp,1500,-82,-62,-86,29,2,4,0.25,-76.277778,2.841024,0.988372,NaN,794.0,0.988372,NaN


In [49]:
test_data.fillna(0, inplace=True)

test_data.replace([np.inf, -np.inf], 0, inplace=True)
test_data

,test_dur,protocol,pkt_len,pd,ed,nav,eirp,nss,mcs,per,mean_rssi_value,std_rssi_value,ap_hear_prob_pd,nav_silence_ratio,pd_counts,ap_pair_hear_prob_pd,throughput
0,60,tcp,1500,-82,-62,-86,9,2,3,0.14,-82.813953,3.696358,0.348837,0.0,79.0,0.293103,0.0
1,60,tcp,1500,-82,-62,-86,9,2,4,0.13,-84.086207,4.260017,0.293103,0.0,42.0,0.293103,0.0
2,60,tcp,1500,-82,-62,-86,9,2,4,0.15,-79.225000,4.803579,0.825000,0.0,41.0,0.293103,0.0
3,60,udp,1500,-82,-62,-86,9,1,5,0.15,-82.813953,3.696358,0.348837,0.0,79.0,0.293103,0.0
4,60,udp,1500,-82,-62,-86,9,2,4,0.13,-84.086207,4.260017,0.293103,0.0,42.0,0.293103,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,60,tcp,1500,-82,-62,-86,29,2,11,0.19,-76.277778,2.841024,0.988372,0.0,794.0,0.988372,0.0
101,60,tcp,1500,-82,-62,-86,29,2,11,0.15,-73.325758,2.142627,0.997475,0.0,800.0,0.988372,0.0
102,60,udp,1500,-82,-62,-86,29,2,8,0.33,-74.237221,3.802473,1.000000,0.0,789.0,0.988372,0.0
103,60,udp,1500,-82,-62,-86,29,2,4,0.25,-76.277778,2.841024,0.988372,0.0,794.0,0.988372,0.0


In [50]:
test_data[features]
test_data = pd.get_dummies(test_data, columns=['protocol'], drop_first=True)
# test_data.drop('seq_time', index=1)

In [51]:
test_data.drop('throughput', axis=1)

,test_dur,pkt_len,pd,ed,nav,eirp,nss,mcs,per,mean_rssi_value,std_rssi_value,ap_hear_prob_pd,nav_silence_ratio,pd_counts,ap_pair_hear_prob_pd,protocol_udp
0,60,1500,-82,-62,-86,9,2,3,0.14,-82.813953,3.696358,0.348837,0.0,79.0,0.293103,0
1,60,1500,-82,-62,-86,9,2,4,0.13,-84.086207,4.260017,0.293103,0.0,42.0,0.293103,0
2,60,1500,-82,-62,-86,9,2,4,0.15,-79.225000,4.803579,0.825000,0.0,41.0,0.293103,0
3,60,1500,-82,-62,-86,9,1,5,0.15,-82.813953,3.696358,0.348837,0.0,79.0,0.293103,1
4,60,1500,-82,-62,-86,9,2,4,0.13,-84.086207,4.260017,0.293103,0.0,42.0,0.293103,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,60,1500,-82,-62,-86,29,2,11,0.19,-76.277778,2.841024,0.988372,0.0,794.0,0.988372,0
101,60,1500,-82,-62,-86,29,2,11,0.15,-73.325758,2.142627,0.997475,0.0,800.0,0.988372,0
102,60,1500,-82,-62,-86,29,2,8,0.33,-74.237221,3.802473,1.000000,0.0,789.0,0.988372,1
103,60,1500,-82,-62,-86,29,2,4,0.25,-76.277778,2.841024,0.988372,0.0,794.0,0.988372,1


In [52]:
test_data.columns

Index(['test_dur', 'pkt_len', 'pd', 'ed', 'nav', 'eirp', 'nss', 'mcs', 'per',
       'mean_rssi_value', 'std_rssi_value', 'ap_hear_prob_pd',
       'nav_silence_ratio', 'pd_counts', 'ap_pair_hear_prob_pd', 'throughput',
       'protocol_udp'],
      dtype='object')

In [54]:
test_prediction = rf_model.predict(test_data.drop('throughput', axis=1))
test_prediction_100 = test_prediction + 100

In [55]:
test_prediction_100

array([159.7634 , 166.10543, 162.44032, 159.74821, 166.37489, 160.92577,
       163.79445, 164.23223, 161.08724, 164.08229, 162.49537, 164.21194,
       161.52337, 167.45602, 165.54719, 160.8618 , 165.83002, 163.61304,
       181.8031 , 167.46396, 160.66399, 182.2953 , 167.76179, 162.48869,
       175.59906, 168.59327, 195.6883 , 167.4934 , 172.47867, 179.17263,
       209.32535, 167.48365, 203.34928, 210.43906, 167.91161, 167.53951,
       208.44439, 160.51454, 201.51174, 176.88327, 161.36175, 189.28181,
       180.46859, 161.0803 , 172.6485 , 197.63981, 163.01327, 189.35795,
       164.60932, 165.65369, 169.52664, 185.3949 , 163.60413, 170.38751,
       164.02484, 159.58191, 174.04195, 161.22226, 170.58833, 172.4391 ,
       161.6531 , 163.47987, 169.76163, 163.52511, 168.99449, 199.58405,
       163.32548, 163.77023, 168.80123, 166.29443, 167.91831, 167.55588,
       162.46392, 172.26329, 171.41507, 160.43479, 170.55994, 187.14323,
       161.82388, 173.11096, 169.01915, 159.52326, 

In [135]:
import pandas as pd
import numpy as np
import ast
from sklearn.cluster import KMeans

def process_file(file_path):
    # 读取数据
    data = pd.read_csv(file_path)

    # 定义需要处理的列
    rssi_columns = [
        'ap_from_ap_0_sum_ant_rssi',
        'ap_from_ap_0_max_ant_rssi',
        'ap_from_ap_0_mean_ant_rssi',
        'ap_from_ap_1_sum_ant_rssi',
        'ap_from_ap_1_max_ant_rssi',
        'ap_from_ap_1_mean_ant_rssi',
        'ap_from_ap_2_sum_ant_rssi', 
        'ap_from_ap_2_max_ant_rssi',
        'ap_from_ap_2_mean_ant_rssi',
        'sta_to_ap_0_sum_ant_rssi',
        'sta_to_ap_0_max_ant_rssi',
        'sta_to_ap_0_mean_ant_rssi',
        'sta_to_ap_1_sum_ant_rssi',
        'sta_to_ap_1_max_ant_rssi',
        'sta_to_ap_1_mean_ant_rssi',
        'sta_to_ap_2_sum_ant_rssi', 
        'sta_to_ap_2_max_ant_rssi',
        'sta_to_ap_2_mean_ant_rssi',
        'sta_from_ap_0_sum_ant_rssi',
        'sta_from_ap_0_max_ant_rssi',
        'sta_from_ap_0_mean_ant_rssi',
        'sta_from_ap_1_sum_ant_rssi',
        'sta_from_ap_1_max_ant_rssi',
        'sta_from_ap_1_mean_ant_rssi',
        'sta_from_ap_2_sum_ant_rssi',
        'sta_from_ap_2_max_ant_rssi', 
        'sta_from_ap_2_mean_ant_rssi'
    ]

    # 将字符串转换为浮点数列表
    def str_to_float_list(s):
        try:
            return [float(x) for x in ast.literal_eval(s)]
        except:
            return []

    for col in rssi_columns:
        data[f"{col}_list"] = data[col].apply(str_to_float_list)

    grouped_data = data.groupby('test_id')

    # 定义用于去噪的函数
    def kmeans_filter_rssi(rssi_list, n_clusters=2):
        if len(rssi_list) <= 1:
            return rssi_list
        rssi_array = np.array(rssi_list).reshape(-1, 1)
        kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
        labels = kmeans.labels_
        centers = kmeans.cluster_centers_
        if centers[0] < centers[1]:
            noise_label = 0
        else:
            noise_label = 1
        filtered_rssi = [rssi for rssi, label in zip(rssi_list, labels) if label != noise_label]
        return filtered_rssi

    # 对每个测试ID的每个RSSI列进行去噪处理
    for test_id, group in grouped_data:
        if len(group) != 3:
            print(f"测试ID {test_id} 的数据行数不是3行，实际为 {len(group)} 行。请检查数据。")
            continue  # 跳过这个测试
        for col in rssi_columns:
            for idx, row in group.iterrows():
                clean_rssi = kmeans_filter_rssi(row[f"{col}_list"])
                data.at[idx, f"{col}_list"] = clean_rssi

    # 定义计算互听概率的函数
    def calculate_hearing_probability(rssi_list, threshold):
        if not rssi_list:
            return 0
        above_threshold = [rssi for rssi in rssi_list if rssi >= threshold]
        return len(above_threshold) / len(rssi_list)

    # 计算每个AP的互听概率
    for test_id, group in grouped_data:
        if len(group) != 3:
            continue  # 已经在上面提示过，这里直接跳过
        ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}
        ap_ids = list(ap_rows.keys())  # ['ap_0', 'ap_1', 'ap_2']

        for ap_id in ap_ids:
            current_row = ap_rows[ap_id]
            current_ap_index = int(ap_id.split('_')[1])
            other_ap_indices = [i for i in range(3) if i != current_ap_index]
            combined_rssi_list = []

            for other_ap_index in other_ap_indices:
                col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
                rssi_list = current_row[col_name]

                if isinstance(rssi_list, list):
                    combined_rssi_list.extend(rssi_list)

            pd_threshold = current_row['pd']
            hearing_probability = calculate_hearing_probability(combined_rssi_list, pd_threshold)
            data.at[current_row.name, 'ap_hear_prob_pd'] = hearing_probability

        hearing_probs = [data.at[ap_rows[ap_id].name, 'ap_hear_prob_pd'] for ap_id in ap_ids]
        overall_min_hear_prob = min(hearing_probs)

        for ap_id in ap_ids:
            data.at[ap_rows[ap_id].name, 'ap_pair_hear_prob_pd'] = overall_min_hear_prob

    # 定义辅助函数
    def can_hear_each_other(rssi_list, threshold):
        if not rssi_list or not isinstance(rssi_list, list):
            return False
        return any(rssi >= threshold for rssi in rssi_list)

    def calculate_nav_period(seq_time):
        # 您需要根据实际的NAV计算逻辑来实现这个函数
        # 这里我们假设NAV周期等于seq_time
        return seq_time

    # 计算每个AP的NAV静默时间和比例
    for test_id, group in grouped_data:
        if len(group) != 3:
            continue
        ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}
        nav_threshold = group['nav'].iloc[0]
        test_duration = group['test_dur'].iloc[0] * 1e6  # 将秒转换为微秒

        for ap_id, current_row in ap_rows.items():
            current_ap_index = int(ap_id.split('_')[1])
            current_nav_silence_time = 0
            other_ap_ids = [other_id for other_id in ap_rows.keys() if other_id != ap_id]

            for other_ap_id in other_ap_ids:
                other_row = ap_rows[other_ap_id]
                other_ap_index = int(other_ap_id.split('_')[1])
                col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
                rssi_list = current_row[col_name]

                can_hear = can_hear_each_other(rssi_list, nav_threshold)

                if can_hear:
                    seq_time_other = other_row['seq_time'] * 1e6  # 将秒转换为微秒
                    nav_period = calculate_nav_period(seq_time_other)
                    num_transmissions_other = other_row['num_ampdu']
                    nav_silence_time = nav_period * num_transmissions_other
                    current_nav_silence_time += nav_silence_time

            if test_duration != 0:
                nav_silence_ratio = current_nav_silence_time / test_duration
            else:
                nav_silence_ratio = 0

            data.at[current_row.name, f'{ap_id}_nav_silence_time'] = current_nav_silence_time
            data.at[current_row.name, f'{ap_id}_nav_silence_ratio'] = nav_silence_ratio

    # 计算每个AP的PD计数
    def calculate_pd_counts(rssi_list, pd_threshold):
        if not rssi_list or not isinstance(rssi_list, list):
            return 0
        pd_counts = sum(1 for rssi in rssi_list if rssi >= pd_threshold)
        return pd_counts

    for test_id, group in grouped_data:
        if len(group) != 3:
            continue
        ap_rows = {row['ap_id']: row for idx, row in group.iterrows()}
        pd_counts_dict = {}

        for ap_id, current_row in ap_rows.items():
            current_ap_index = int(ap_id.split('_')[1])
            pd_threshold = current_row['pd']
            combined_rssi_list = []
            other_ap_indices = [i for i in range(3) if i != current_ap_index]

            for other_ap_index in other_ap_indices:
                col_name = f'ap_from_ap_{other_ap_index}_max_ant_rssi_list'
                rssi_list = current_row[col_name]

                if isinstance(rssi_list, list):
                    combined_rssi_list.extend(rssi_list)

            pd_counts = calculate_pd_counts(combined_rssi_list, pd_threshold)
            data.at[current_row.name, f'{ap_id}_pd_counts'] = pd_counts

    # 计算平均RSSI值和标准差
    def list_mean(lst):
        return sum(lst) / len(lst) if lst else None

    def list_std(lst):
        return np.std(lst) if lst else None

    def get_combined_rssi_list(row):
        current_ap_index = int(row['ap_id'].split('_')[1])
        other_ap_indices = [i for i in range(3) if i != current_ap_index]
        combined_rssi_list = []
        for other_ap_index in other_ap_indices:
            col_name = f'ap_from_ap_{other_ap_index}_mean_ant_rssi_list'
            rssi_list = row[col_name]
            if isinstance(rssi_list, list):
                combined_rssi_list.extend(rssi_list)
        return combined_rssi_list

    data['mean_rssi_value'] = data.apply(lambda row: list_mean(get_combined_rssi_list(row)), axis=1)
    data['std_rssi_value'] = data.apply(lambda row: list_std(get_combined_rssi_list(row)), axis=1)

    # 获取NAV静默时间和比例
    data['nav_silence_time'] = data.apply(lambda row: row.get(f"{row['ap_id']}_nav_silence_time", 0), axis=1)
    data['nav_silence_ratio'] = data.apply(lambda row: row.get(f"{row['ap_id']}_nav_silence_ratio", 0), axis=1)

    # 获取PD计数
    data['pd_counts'] = data.apply(lambda row: row.get(f"{row['ap_id']}_pd_counts", 0), axis=1)

    # 选择需要的特征
    features = [
        'test_dur', 'protocol', 'pkt_len', 'pd', 'ed', 'nav',
        'eirp', 'nss', 'mcs', 'per', 'mean_rssi_value', 'std_rssi_value',
        'ap_hear_prob_pd', 'nav_silence_ratio', 'pd_counts', 'ap_pair_hear_prob_pd',
        'seq_time'  # 如果您需要目标变量
    ]

    # 返回处理后的数据
    return data[features]

# 主程序部分
if __name__ == "__main__":
    # 文件列表
    file_list = [
        "D:\\Data\\研赛建模\\B题\\test_set_2_3ap.csv"
    ]

    # 存储处理后的数据
    processed_data_list = []

    # 处理每个文件
    for file_path in file_list:
        print(f"正在处理文件：{file_path}")
        processed_data = process_file(file_path)
        processed_data_list.append(processed_data)

    # 合并所有数据集
    combined_data = pd.concat(processed_data_list, ignore_index=True)

    # 保存合并后的数据集到CSV文件（可选）
    combined_data.to_csv("test_training_set_2.csv", index=False)

    # 输出处理完成的提示
    print("所有文件已处理完成，合并后的数据集已生成。")


正在处理文件：D:\Data\研赛建模\B题\test_set_2_3ap.csv


C:\Users\ROG\AppData\Local\Temp\ipykernel_44444\2927560338.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_44444\2927560338.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_44444\2927560338.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(rssi_array)
C:\Users\ROG\AppData\Local\Temp\ipykernel_44444\2927560338.py:58: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  kmeans = KM

所有文件已处理完成，合并后的数据集已生成。


In [147]:
test_data_2 = pd.read_csv("D:\\Data\\研赛建模\\B题\\test_training_set_2.csv")

In [148]:
test_data_2.fillna(0, inplace=True)

test_data_2.replace([np.inf, -np.inf], 0, inplace=True)
test_data_2

,test_dur,protocol,pkt_len,pd,ed,nav,eirp,nss,mcs,per,mean_rssi_value,std_rssi_value,ap_hear_prob_pd,nav_silence_ratio,pd_counts,ap_pair_hear_prob_pd,seq_time
0,60,tcp,1500,-82,-62,-82,9,0.0,0.0,0.0,-88.647887,4.052815,0.084507,0.0,20.0,0.084507,0.0
1,60,tcp,1500,-82,-62,-82,9,0.0,0.0,0.0,-79.774194,9.022229,0.612903,0.0,31.0,0.084507,0.0
2,60,tcp,1500,-82,-62,-82,9,0.0,0.0,0.0,-72.886364,5.104436,1.000000,0.0,45.0,0.084507,0.0
3,60,udp,1500,-82,-62,-82,9,0.0,0.0,0.0,-88.647887,4.052815,0.084507,0.0,20.0,0.084507,0.0
4,60,udp,1500,-82,-62,-82,9,0.0,0.0,0.0,-79.774194,9.022229,0.612903,0.0,31.0,0.084507,0.0
5,60,udp,1500,-82,-62,-82,9,0.0,0.0,0.0,-72.886364,5.104436,1.000000,0.0,45.0,0.084507,0.0
6,60,tcp,1500,-82,-62,-82,10,0.0,0.0,0.0,-69.000000,6.442049,1.000000,0.0,22.0,0.113924,0.0
7,60,tcp,1500,-82,-62,-82,10,0.0,0.0,0.0,-89.160338,4.280220,0.113924,0.0,45.0,0.113924,0.0
8,60,tcp,1500,-82,-62,-82,10,0.0,0.0,0.0,-74.867925,4.778244,1.000000,0.0,59.0,0.113924,0.0
9,60,udp,1500,-82,-62,-82,10,0.0,0.0,0.0,-69.000000,6.442049,1.000000,0.0,22.0,0.113924,0.0


In [149]:
test_data_2[features]
test_data_2 = pd.get_dummies(test_data_2[features], columns=['protocol'], drop_first=True)

In [152]:
test_data_2.columns

Index(['test_dur', 'pkt_len', 'pd', 'ed', 'nav', 'eirp', 'nss', 'mcs', 'per',
       'mean_rssi_value', 'std_rssi_value', 'ap_hear_prob_pd',
       'nav_silence_ratio', 'pd_counts', 'ap_pair_hear_prob_pd',
       'protocol_udp'],
      dtype='object')

In [154]:
test_data_2[features_2].columns

Index(['test_dur', 'pkt_len', 'pd', 'ed', 'nav', 'eirp', 'per',
       'mean_rssi_value', 'std_rssi_value', 'nav_silence_ratio', 'pd_counts',
       'ap_pair_hear_prob_pd', 'protocol_udp'],
      dtype='object')

In [155]:
test_predictions_2 = rf_classifier.predict(test_data_2[features_2])

In [156]:
test_predictions_2


array([19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
       19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
       19, 19])